In [9]:
import os
import pandas as pd
import tensorflow as tf
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Bidirectional, Conv1D, GlobalMaxPooling1D, Concatenate, AdditiveAttention
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import AdditiveAttention
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from utils import eval, eval_training, get_train_test_data, save_for_evaluation, process_train_test_data
import re
import numpy as np
import spacy
from gensim.models import Word2Vec
from transformers import BertTokenizer, TFBertModel


os.environ['TF_NUM_INTEROP_THREADS'] = '10'
os.environ['TF_NUM_INTRAOP_THREADS'] = '10'

# Configure TensorFlow session for multi-threading
tf.config.threading.set_inter_op_parallelism_threads(10)
tf.config.threading.set_intra_op_parallelism_threads(10)
# Ensure TensorFlow is using the Metal backend
gpu = len (tf.config.list_physical_devices ('GPU'))>0
print ("GPU is", "available" if gpu else "NOT AVAILABLE")

GPU is available


# Data preparation

In [10]:
data_label = "data"
predict_label = "label"
balanced = False
labels = None
class_weights = False
sample_weights = False
df_train = pd.read_csv('../data/data_train.csv')
df_valid = pd.read_csv('../data/data_valid.csv')
df_test = pd.read_csv('../data/data_test.csv')

train_texts = df_train[data_label]
valid_texts = df_valid[data_label]
test_texts = df_test[data_label]

In [11]:
# Initialize the label encoder
all_labels = pd.concat([df_train[predict_label], df_valid[predict_label], df_test[predict_label]])
label_encoder = LabelEncoder()
label_encoder.fit(all_labels)
# Fit label encoder and return encoded labels as integers
train_labels_enc = label_encoder.transform(df_train[predict_label])
valid_labels_enc = label_encoder.transform(df_valid[predict_label])
test_labels_enc = label_encoder.transform(df_test[predict_label])

# Convert labels to categorical one-hot encoding
train_labels = to_categorical(train_labels_enc)
valid_labels = to_categorical(valid_labels_enc)
test_labels = to_categorical(test_labels_enc)

num_classes = len(df_train[predict_label].unique())

In [12]:
# Load spaCy model
nlp = spacy.load('de_core_news_sm')

def preprocess_text(text):
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-zA-ZäöüÄÖÜß\s]', '', text)
    doc = nlp(text)
    stop_words = spacy.lang.de.stop_words.STOP_WORDS
    words = [token.text for token in doc if token.text.lower() not in stop_words and token.is_alpha]
    return words

def get_word2vec_embeddings(text, model, vector_size=300):
    tokens = preprocess_text(text)
    embeddings = np.zeros((len(tokens), vector_size))
    for i, token in enumerate(tokens):
        if token in model.wv:
            embeddings[i] = model.wv[token]
        else:
            embeddings[i] = np.zeros(vector_size)
    return embeddings

# Load pre-trained Word2Vec embeddings
w2v = Word2Vec.load("../data/word2vec.model")

w2v_train_embeddings = [get_word2vec_embeddings(text, w2v) for text in train_texts]
w2v_valid_embeddings = [get_word2vec_embeddings(text, w2v) for text in valid_texts]
w2v_test_embeddings = [get_word2vec_embeddings(text, w2v) for text in test_texts]
max_length = max(len(embedding) for embedding in w2v_train_embeddings)
w2v_padded_train_embeddings = pad_sequences(w2v_train_embeddings, maxlen=max_length, padding='post', dtype='float32')
w2v_padded_valid_embeddings = pad_sequences(w2v_valid_embeddings, maxlen=max_length, padding='post', dtype='float32')
w2v_padded_test_embeddings = pad_sequences(w2v_test_embeddings, maxlen=max_length, padding='post', dtype='float32')

# Model parameters

In [13]:
embedding_dim = 300
lstm_units = 64
epochs = 15
batch_size = 32
model_name = "word2vec"

In [21]:
optimizer = Adam()
loss = tf.keras.losses.CategoricalCrossentropy()
metrics=[
      tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.F1Score(average="macro", name='f1_score'),
]
early_stopping_callback = EarlyStopping(
    monitor='val_f1_score',  
    patience=3,         
    restore_best_weights=True,
    verbose=1
)

# Model definition and training

In [22]:
# Define the input layer
input_text = Input(shape=(max_length, embedding_dim), dtype='float32', name='text_input')

# Two LSTM layers
x = Bidirectional(LSTM(units=lstm_units, return_sequences=True))(input_text)
x = Dropout(0.5)(x)
x = Bidirectional(LSTM(units=lstm_units))(x)

# Output layer
output = Dense(num_classes, activation='softmax')(x)

# Define the model
model = Model(inputs=input_text, outputs=output)

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [23]:
history = model.fit(w2v_padded_train_embeddings, train_labels, epochs=epochs, batch_size=batch_size,
                    validation_data=(w2v_padded_valid_embeddings, valid_labels), callbacks=[early_stopping_callback])

Epoch 1/10
736/736 ━━━━━━━━━━━━━━━━━━━━ 95s 127ms/step - accuracy: 0.4262 - auc: 0.7879 - f1_score: 0.2061 - loss: 1.2251 - val_accuracy: 0.4776 - val_auc: 0.8246 - val_f1_score: 0.2106 - val_loss: 1.1321
Epoch 2/10
736/736 ━━━━━━━━━━━━━━━━━━━━ 92s 125ms/step - accuracy: 0.4856 - auc: 0.8287 - f1_score: 0.2483 - loss: 1.1199 - val_accuracy: 0.4986 - val_auc: 0.8334 - val_f1_score: 0.3007 - val_loss: 1.1110
Epoch 3/10
736/736 ━━━━━━━━━━━━━━━━━━━━ 91s 124ms/step - accuracy: 0.5160 - auc: 0.8377 - f1_score: 0.3005 - loss: 1.1026 - val_accuracy: 0.5005 - val_auc: 0.8381 - val_f1_score: 0.3016 - val_loss: 1.0996
Epoch 4/10
736/736 ━━━━━━━━━━━━━━━━━━━━ 93s 127ms/step - accuracy: 0.5262 - auc: 0.8461 - f1_score: 0.3129 - loss: 1.0754 - val_accuracy: 0.5069 - val_auc: 0.8402 - val_f1_score: 0.3110 - val_loss: 1.0919
Epoch 5/10
736/736 ━━━━━━━━━━━━━━━━━━━━ 92s 124ms/step - accuracy: 0.5367 - auc: 0.8499 - f1_score: 0.3206 - loss: 1.0651 - val_accuracy: 0.5228 - val_auc: 0.8444 - val_f1_score: 0

In [ ]:
save_for_evaluation(model, history, model_name, w2v_padded_test_embeddings, test_labels, label_encoder)

In [ ]:
eval_training(history, metrics)

In [ ]:
eval(model, w2v_padded_test_embeddings, test_labels, label_encoder)